In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
train=pd.read_csv("/kaggle/input/iitmpaybuddy-g1/train.csv", index_col=0)

In [4]:
test=pd.read_csv("/kaggle/input/iitmpaybuddy-g1/test.csv", index_col=0)

## Dropping columns with missing values > 50%

In [5]:
empty_columns = []
for column in train.columns:
  if((train[column].isnull().sum()/train.shape[0])>0.5):
    empty_columns.append(column)
empty_columns

In [6]:
train.drop(columns=empty_columns, inplace=True)
test.drop(columns=empty_columns, inplace=True)

# Imputation

In [7]:
train.fillna(train.mean(), inplace=True)

In [8]:
test.fillna(test.mean(), inplace=True)

# Training Models

In [9]:
y = train['target'].astype(int)
X = train.drop(columns=['target', 'cust_id'])

# XGBoost

In [ ]:
!pip install xgboost

In [10]:
import xgboost as xgb

In [11]:
# final_train = xgb.DMatrix(X, label=y)

In [12]:
param = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 3}  # the number of classes that exist in this datset
num_round = 20  # the number of training iterations

In [12]:
xgb_model_mpg = xgb.XGBClassifier(max_depth= 3, n_estimators=100, eta=0.3)

In [13]:
xgb_model_mpg.fit(X, y)

In [15]:
test

In [20]:
res = xgb_model_mpg.predict(test.drop('cust_id', axis=1))

In [28]:
probs = xgb_model_mpg.predict_proba(test.drop('cust_id', axis=1))

In [31]:
probs[:, 0]

# Submission File

In [40]:
test["preds"] = probs[:, 1]

In [41]:
final_sub = test[["cust_id", "preds"]]

In [42]:
final_sub=final_sub.set_index('cust_id')
final_sub.to_csv("XGB_full.csv")